# DA 350
# Paronamic Tour of Vietnam
# Phineas Pham & Hung Tran
## May 2023

In [ ]:
#import libraries
!pip install gurobipy
import pandas as pd
import gurobipy as gp
from gurobipy import *
from datetime import date
import gurobipy as GRB
#Import packages
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import geopy.distance
import folium
from itertools import combinations, permutations
import itertools
import math
from geopy.distance import great_circle

In [ ]:
# Create an environment with your WLS license
# key values left blank
params = {
"WLSACCESSID": '',
"WLSSECRET": ,
"LICENSEID": ,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
model = gp.Model(env=env)

## Data Preprocessing:

#### Input data:

In [ ]:
df_location = pd.read_csv('/content/location.xlsx - Sheet1.csv')
df_location_noindex = df_location
df_location = df_location.set_index('Tourist Attraction')
df_location

,City,Region,Star Rating,Latitude,Longitude
Tourist Attraction,,,,,
Hoan Kiem Lake,Hanoi,North,4.5,21.027846,105.852278
Long Bien Bridge,Hanoi,North,4.0,21.044483,105.860837
Ha Long Bay,Quang Ninh,North,4.5,20.971205,107.049051
Hoi An Ancient Town,Quang Nam,Central,4.5,15.880301,108.338008
Notre Dame Square,Ho Chi Minh City,South,4.0,10.779965,106.699105
Cu Chi Tunnel,Ho Chi Minh City,South,4.5,11.028019,106.512466
Thap Tram Huong,Nha Trang,Central,3.5,12.240540,109.196901
Dragon Bridge,Da Nang,Central,4.0,16.061336,108.227517
Cai Rang Floating Market,Can Tho,South,4.0,10.005237,105.745939


In [ ]:
df_location.loc['Hoan Kiem Lake']['Star Rating']  # == 4.5

4.5

#### initiate attraction map:

In [ ]:
map_museum = folium.Map(location=[16,110],
                 zoom_start= 5.5,
                 control_scale=True)

for i,row in df_location_noindex.iterrows():
    #if i != 1:
    iframe = folium.IFrame('Attraction Name:' + str(row["Tourist Attraction"]))

    #Initialise the popup using the iframe
    popup = folium.Popup(iframe, min_width=300, max_width=300)

    #Add each row to the map
    folium.Marker(location=[row['Latitude'],row['Longitude']],
                popup = popup, c=row['Tourist Attraction']).add_to(map_museum)

folium.Marker(location=[df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == 'Notre Dame Square', "Latitude"].iloc[0],
                        df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == "Notre Dame Square", "Longitude"].iloc[0]],
              icon=folium.Icon(color='red', icon='pushpin')).add_to(map_museum)

map_museum

### Traveling Salesman Estimate Solution: (by Gurobi provider)
Functions that will generate constraints to solve the TS problem

In [ ]:
## Calculate shortest subtour on a route
def subtour(edges):
    nodes = set(i for e in edges for i in e)
    unvisited = list(nodes)
    cycle = list(nodes)
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

## Subtour elimination constraint generator
def subtourelim(model, where):
    global subtour_iterations
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
            if vals[i, j] > 0.5)
        tour = subtour(selected)
        if len(tour) < len(selected): #len(selected) is total number of edges
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in permutations(tour, 2)) <= len(tour)-1)

input cost and time dataframes, then store in dict_cost and dict_time:

In [ ]:
df_cost = pd.read_csv('/content/cost_df - Sheet1.csv')
df_cost = df_cost.fillna(999) #make na values to 999 so that model will not pick unavailable options
df_cost

,Departure,Arrival,Walking,Bus,Taxi,Train,Airplane
0,Hoan Kiem Lake,Long Bien Bridge,0,999.0,2.0,999.0,999.0
1,Hoan Kiem Lake,Ha Long Bay,0,20.0,115.0,999.0,263.0
2,Hoan Kiem Lake,Hoi An Ancient Town,0,34.0,999.0,35.5,65.5
3,Hoan Kiem Lake,Notre Dame Square,0,55.0,999.0,55.0,102.5
4,Hoan Kiem Lake,Cu Chi Tunnel,0,94.0,999.0,80.0,127.0
...,...,...,...,...,...,...,...
61,Cai Rang Floating Market,Fansipan Mountain,0,81.0,999.0,83.5,253.0
62,Cai Rang Floating Market,Hue Imperial City,0,46.0,999.0,60.5,314.5
63,Vung Tau Lighthouse,Fansipan Mountain,0,82.0,999.0,79.5,128.0
64,Vung Tau Lighthouse,Hue Imperial City,0,28.0,999.0,92.0,221.5


In [ ]:

dict_cost = {}
for i in range(66):
  temp = []
  temp.append(df_cost.at[i, 'Walking'])
  temp.append(df_cost.at[i, 'Bus'])
  temp.append(df_cost.at[i, 'Taxi'])
  temp.append(df_cost.at[i, 'Train'])
  temp.append(df_cost.at[i, 'Airplane'])
  dict_cost[(df_cost.at[i, 'Departure'], df_cost.at[i, 'Arrival'])] = temp
  dict_cost[(df_cost.at[i, 'Arrival'], df_cost.at[i, 'Departure'])] = temp
#dict_cost

In [ ]:
df_time = pd.read_csv('/content/time_df - Sheet1.csv')
df_time = df_time.fillna(999)  #make na values as 999 so that the model won't choose these options
df_time

,Departure,Arrival,Walking,Bus,Taxi,Train,Airplane
0,Hoan Kiem Lake,Long Bien Bridge,0.5,999.00,0.067,999.00,999.00
1,Hoan Kiem Lake,Ha Long Bay,999.0,3.15,1.670,999.00,2.75
2,Hoan Kiem Lake,Hoi An Ancient Town,999.0,17.00,999.000,17.00,4.24
3,Hoan Kiem Lake,Notre Dame Square,999.0,35.85,32.550,999.00,4.40
4,Hoan Kiem Lake,Cu Chi Tunnel,999.0,35.60,999.000,32.55,4.33
...,...,...,...,...,...,...,...
61,Cai Rang Floating Market,Fansipan Mountain,999.0,47.00,999.000,44.60,6.45
62,Cai Rang Floating Market,Hue Imperial City,999.0,28.60,999.000,24.50,5.30
63,Vung Tau Lighthouse,Fansipan Mountain,999.0,41.75,999.000,42.80,9.80
64,Vung Tau Lighthouse,Hue Imperial City,999.0,25.00,999.000,20.30,5.40


In [ ]:

dict_time = {}
for i in range(66):
  temp = []
  temp.append(df_time.at[i, 'Walking'])
  temp.append(df_time.at[i, 'Bus'])
  temp.append(df_time.at[i, 'Taxi'])
  temp.append(df_time.at[i, 'Train'])
  temp.append(df_time.at[i, 'Airplane'])
  dict_time[(df_time.at[i, 'Departure'], df_time.at[i, 'Arrival'])] = temp
  dict_time[(df_time.at[i, 'Arrival'], df_time.at[i, 'Departure'])] = temp
#dict_time

## Optimal model:

In [ ]:
time_per_place = 5
_Time = 100
_Cost = 400


mod = gp.Model("Paranomic Tour")
mod.Params.LogToConsole = 0
location = list(df_location.index)
n = len(df_cost.keys())  #number of pairs

#add binary variables
a = mod.addVars(dict_cost.keys(), vtype="B", name = "walk")
b = mod.addVars(dict_cost.keys(), vtype="B", name = "bus")
c = mod.addVars(dict_cost.keys(), vtype="B", name = "taxi")
d = mod.addVars(dict_cost.keys(), vtype="B", name = "train")
e = mod.addVars(dict_cost.keys(), vtype="B", name = "airplane")

y = mod.addVars(dict_cost.keys(), vtype="B", name = "pathExist")
x = mod.addVars(location, vtype="B") #if visit

# Objective
mod.setObjective(gp.quicksum(df_location.loc[i]['Star Rating'] * x[i] for i in location), GRB.MAXIMIZE)

# Constraints
start = 'Hoan Kiem Lake'

# Ensure that all locations are visited once
# for i in location:
#     #must leave i once
#     mod.addConstr(gp.quicksum(y[i, j] for j in location if j != i) == 1)
#     #must arrive i once
#     mod.addConstr(gp.quicksum(y[j, i] for j in location if j != i) == 1)


# CONSTRAINTS
for i in location:
    for j in location:
        if i != j:

            mod.addConstr(y[i,j] + y[j,i] <= 1)
            # choose only 1 transportation
            mod.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] <= 1)
            # Ensure that each trip goes in one direction only
            mod.addConstr(y[i,j] + y[j,i] <= x[j])
            # y = a+b+c+d+e constraint
            mod.addConstr(y[i,j] == (a[i,j]+b[i,j]+c[i,j]+d[i,j]+e[i,j]))
            # one transportation btwn 2 locations
            mod.addConstr(y[i,j] <= 1)
            mod.addConstr(a[i,j]+b[i,j]+c[i,j]+d[i,j]+e[i,j] <= 1)

for i in location:
    if i != start:
        # x constraint
        mod.addConstr(gp.quicksum(y[i, j] for j in location if j != i) == x[i])
        mod.addConstr(gp.quicksum(y[j, i] for j in location if j != i) == x[i])
        mod.addConstr(gp.quicksum(y[i, j] for j in location if j != i) <= 1)
        mod.addConstr(gp.quicksum(y[j, i] for j in location if j != i) <= 1)

        mod.addConstr(gp.quicksum(y[i, j] for j in location if j != i) == gp.quicksum(y[j, i] for j in location if j != i))

#sum go to = leave
#mod.addConstr()

#TIME constraint
mod.addQConstr(gp.quicksum(a[i,j]*dict_time[i,j][0] + b[i,j]*dict_time[i,j][1] + c[i,j]*dict_time[i,j][2] + d[i,j]*dict_time[i,j][3] + e[i,j]*dict_time[i,j][4] for (i,j) in dict_cost.keys()) + gp.quicksum(time_per_place*x[j] for j in location) <= _Time) #24 cuz each pair counts one location twice
#COST constraint
mod.addQConstr(gp.quicksum(a[i,j]*dict_cost[i,j][0] + b[i,j]*dict_cost[i,j][1] + c[i,j]*dict_cost[i,j][2] + d[i,j]*dict_cost[i,j][3] + e[i,j]*dict_cost[i,j][4] for (i,j) in dict_cost.keys()) <= _Cost)



# Optimize
mod._vars = y
# mod._vars = a
# mod._vars = b
# mod._vars = c
# mod._vars = d
# mod._vars = e
mod.Params.lazyConstraints = 1
mod.optimize(subtourelim)


if mod.Status == gp.GRB.INFEASIBLE:
    print("Model is infeasible")
elif mod.Status == gp.GRB.UNBOUNDED:
    print("Model is unbounded")
else:
    # Print results
    print("Objective Value:", mod.ObjVal)
    #print("Time taken:", round(mod.ObjVal * 60), "hours")
    print("----------------------------------")
    print("Optimal Route:")
    tour = ['Hoan Kiem Lake']
    # a_ls = ['Hoan Kiem Lake']
    # b_ls = ['Hoan Kiem Lake']
    # c_ls = ['Hoan Kiem Lake']
    # d_ls = ['Hoan Kiem Lake']
    # e_ls = ['Hoan Kiem Lake']
    current = 'Hoan Kiem Lake'
    count = 0
    while True:
        count  += 1
        for j in location:
            if j != current and y[current, j].X > 0.5:
                tour.append(j)
                current = j
                count += 1
                break

        if current == 'Hoan Kiem Lake':
            break
        if count == 100:
          break


    print(" -> ".join(tour))
    print("----------------------------------")
    print("# of locations", count)
    #vals = mod.getAttr('x', y)
    #selected = gp.tuplelist((i, j) for i, j in dict_cost.keys() if y[i, j] > 0.5)
    # for i in range(len(tour)-1):
    #     print(tour[i], "->", tour[i+1], ":", round(dict_cost[tour[i], tour[i+1]],2))


Objective Value: 47.0
----------------------------------
Optimal Route:
Hoan Kiem Lake -> Fansipan Mountain -> Long Bien Bridge -> Ha Long Bay -> Cu Chi Tunnel -> Cai Rang Floating Market -> Notre Dame Square -> Vung Tau Lighthouse -> Dragon Bridge -> Hoi An Ancient Town -> Hue Imperial City -> Hoan Kiem Lake
----------------------------------
# of locations 22


In [ ]:
map_museum = folium.Map(location=[16,110],
                 zoom_start= 5.5,
                 control_scale=True)

for i,row in df_location_noindex.iterrows():
    #if i != 1:
    iframe = folium.IFrame('Attraction Name:' + str(row["Tourist Attraction"]))

    #Initialise the popup using the iframe
    popup = folium.Popup(iframe, min_width=300, max_width=300)

    #Add each row to the map
    folium.Marker(location=[row['Latitude'],row['Longitude']],
                popup = popup, c=row['Tourist Attraction']).add_to(map_museum)

folium.Marker(location=[df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == 'Hoan Kiem Lake', "Latitude"].iloc[0],
                        df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == 'Hoan Kiem Lake', "Longitude"].iloc[0]],
              icon=folium.Icon(color='red', icon='pushpin')).add_to(map_museum)

points = []
for i in tour:
    points.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i, "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i, "Longitude"].iloc[0]])
#add lines
for i in tour:
    folium.PolyLine(points, color="green", weight=2.5, opacity=1).add_to(map_museum)

total_cost = 0
total_time = 0

for i in a:
  if a[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="red", weight=2.5, opacity=1).add_to(map_museum)
    total_time += a[i].X*dict_time[i][0]
    total_cost += a[i].X*dict_cost[i][0]

for i in b:
  if b[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="yellow", weight=2.5, opacity=1).add_to(map_museum)
    total_time += b[i].X*dict_time[i][1]
    total_cost += b[i].X*dict_cost[i][1]

for i in c:
  if c[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="blue", weight=2.5, opacity=1).add_to(map_museum)
    total_time += c[i].X*dict_time[i][2]
    total_cost += c[i].X*dict_cost[i][2]

for i in d:
  if d[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="pink", weight=2.5, opacity=1).add_to(map_museum)
    total_time += d[i].X*dict_time[i][3]
    total_cost += d[i].X*dict_cost[i][3]

for i in e:
  if e[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="cyan", weight=2.5, opacity=1).add_to(map_museum)
    total_time += e[i].X*dict_time[i][4]
    total_cost += e[i].X*dict_cost[i][4]

map_museum

## All location Model:
#### Add constraint to make model visit all inputted locations with relieved time and cost constraints

In [ ]:
time_per_place = 5
_Time = 120
_Cost = 400


mod = gp.Model("Paranomic Tour")
mod.Params.LogToConsole = 0
location = list(df_location.index)
n = len(df_cost.keys())  #number of pairs

#add binary variables
a = mod.addVars(dict_cost.keys(), vtype="B", name = "walk")
b = mod.addVars(dict_cost.keys(), vtype="B", name = "bus")
c = mod.addVars(dict_cost.keys(), vtype="B", name = "taxi")
d = mod.addVars(dict_cost.keys(), vtype="B", name = "train")
e = mod.addVars(dict_cost.keys(), vtype="B", name = "airplane")

y = mod.addVars(dict_cost.keys(), vtype="B", name = "pathExist")
x = mod.addVars(location, vtype="B") #if visit

# Objective
mod.setObjective(gp.quicksum(df_location.loc[i]['Star Rating'] * x[i] for i in location), GRB.MAXIMIZE)

# Constraints
start = 'Hoan Kiem Lake'

#must leave start once
#mod.addConstr(gp.quicksum(a[start, j]+b[start, j]+c[start, j]+d[start, j]+e[start, j] for j in location if j != start) == 1)
#must arrive at Denison once
#mod.addConstr(gp.quicksum(a[j, start]+b[j, start]+c[j, start]+d[j, start]+e[j, start] for j in location if j != start) == 1)

# Ensure that all locations are visited once
for i in location:
    #must leave i once
    mod.addConstr(gp.quicksum(y[i, j] for j in location if j != i) == 1)
    #must arrive i once
    mod.addConstr(gp.quicksum(y[j, i] for j in location if j != i) == 1)


# CONSTRAINTS
for i in location:
    for j in location:
        if i != j:
            # one direction only
            # mod.addConstr(a[i, j] + a[j, i] <= 1)
            # mod.addConstr(b[i, j] + b[j, i] <= 1)
            # mod.addConstr(c[i, j] + c[j, i] <= 1)
            # mod.addConstr(d[i, j] + d[j, i] <= 1)
            # mod.addConstr(e[i, j] + e[j, i] <= 1)

            mod.addConstr(y[i,j] + y[j,i] <= 1)
            # choose only 1 transportation
            mod.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] <= 1)
            # Ensure that each trip goes in one direction only
            mod.addConstr(y[i,j] + y[j,i] <= x[j])
            # y = a+b+c+d+e constraint
            mod.addConstr(y[i,j] == (a[i,j]+b[i,j]+c[i,j]+d[i,j]+e[i,j]))
            # one transportation btwn 2 locations
            mod.addConstr(y[i,j] <= 1)
            mod.addConstr(a[i,j]+b[i,j]+c[i,j]+d[i,j]+e[i,j] <= 1)

for i in location:
    if i != start:
        # x constraint
        mod.addConstr(gp.quicksum(y[i, j] for j in location if j != i) == x[i])
        mod.addConstr(gp.quicksum(y[j, i] for j in location if j != i) == x[i])
        mod.addConstr(gp.quicksum(y[i, j] for j in location if j != i) <= 1)
        mod.addConstr(gp.quicksum(y[j, i] for j in location if j != i) <= 1)

        mod.addConstr(gp.quicksum(y[i, j] for j in location if j != i) == gp.quicksum(y[j, i] for j in location if j != i))

#sum go to = leave
#mod.addConstr()

#TIME constraint
mod.addQConstr(gp.quicksum(a[i,j]*dict_time[i,j][0] + b[i,j]*dict_time[i,j][1] + c[i,j]*dict_time[i,j][2] + d[i,j]*dict_time[i,j][3] + e[i,j]*dict_time[i,j][4] for (i,j) in dict_cost.keys()) + gp.quicksum(time_per_place*x[j] for j in location) <= _Time) #24 cuz each pair counts one location twice
#COST constraint
mod.addQConstr(gp.quicksum(a[i,j]*dict_cost[i,j][0] + b[i,j]*dict_cost[i,j][1] + c[i,j]*dict_cost[i,j][2] + d[i,j]*dict_cost[i,j][3] + e[i,j]*dict_cost[i,j][4] for (i,j) in dict_cost.keys()) <= _Cost)



# Optimize
mod._vars = y
# mod._vars = a
# mod._vars = b
# mod._vars = c
# mod._vars = d
# mod._vars = e
mod.Params.lazyConstraints = 1
mod.optimize(subtourelim)


if mod.Status == gp.GRB.INFEASIBLE:
    print("Model is infeasible")
elif mod.Status == gp.GRB.UNBOUNDED:
    print("Model is unbounded")
else:
    # Print results
    print("Objective Value:", mod.ObjVal)
    #print("Time taken:", round(mod.ObjVal * 60), "hours")
    print("----------------------------------")
    print("Optimal Route:")
    tour = ['Hoan Kiem Lake']
    # a_ls = ['Hoan Kiem Lake']
    # b_ls = ['Hoan Kiem Lake']
    # c_ls = ['Hoan Kiem Lake']
    # d_ls = ['Hoan Kiem Lake']
    # e_ls = ['Hoan Kiem Lake']
    current = 'Hoan Kiem Lake'
    count = 0
    while True:
        count  += 1
        for j in location:
            if j != current and y[current, j].X > 0.5:
                tour.append(j)
                current = j
                count += 1
                break

        if current == 'Hoan Kiem Lake':
            break
        if count == 100:
          break


    print(" -> ".join(tour))
    print("----------------------------------")
    print("# of locations", count)
    #vals = mod.getAttr('x', y)
    #selected = gp.tuplelist((i, j) for i, j in dict_cost.keys() if y[i, j] > 0.5)
    # for i in range(len(tour)-1):
    #     print(tour[i], "->", tour[i+1], ":", round(dict_cost[tour[i], tour[i+1]],2))


Objective Value: 50.5
----------------------------------
Optimal Route:
Hoan Kiem Lake -> Long Bien Bridge -> Ha Long Bay -> Fansipan Mountain -> Thap Tram Huong -> Cai Rang Floating Market -> Cu Chi Tunnel -> Notre Dame Square -> Vung Tau Lighthouse -> Dragon Bridge -> Hoi An Ancient Town -> Hue Imperial City -> Hoan Kiem Lake
----------------------------------
# of locations 24


In [ ]:
map_museum = folium.Map(location=[16,110],
                 zoom_start= 5.5,
                 control_scale=True)

for i,row in df_location_noindex.iterrows():
    #if i != 1:
    iframe = folium.IFrame('Attraction Name:' + str(row["Tourist Attraction"]))

    #Initialise the popup using the iframe
    popup = folium.Popup(iframe, min_width=300, max_width=300)

    #Add each row to the map
    folium.Marker(location=[row['Latitude'],row['Longitude']],
                popup = popup, c=row['Tourist Attraction']).add_to(map_museum)

folium.Marker(location=[df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == 'Hoan Kiem Lake', "Latitude"].iloc[0],
                        df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == 'Hoan Kiem Lake', "Longitude"].iloc[0]],
              icon=folium.Icon(color='red', icon='pushpin')).add_to(map_museum)

points = []
for i in tour:
    points.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i, "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i, "Longitude"].iloc[0]])
#add lines
for i in tour:
    folium.PolyLine(points, color="green", weight=2.5, opacity=1).add_to(map_museum)

total_cost = 0
total_time = 0

for i in a:
  if a[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="red", weight=2.5, opacity=1).add_to(map_museum)
    total_time += a[i].X*dict_time[i][0]
    total_cost += a[i].X*dict_cost[i][0]

for i in b:
  if b[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="yellow", weight=2.5, opacity=1).add_to(map_museum)
    total_time += b[i].X*dict_time[i][1]
    total_cost += b[i].X*dict_cost[i][1]

for i in c:
  if c[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="blue", weight=2.5, opacity=1).add_to(map_museum)
    total_time += c[i].X*dict_time[i][2]
    total_cost += c[i].X*dict_cost[i][2]

for i in d:
  if d[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="pink", weight=2.5, opacity=1).add_to(map_museum)
    total_time += d[i].X*dict_time[i][3]
    total_cost += d[i].X*dict_cost[i][3]

for i in e:
  if e[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="cyan", weight=2.5, opacity=1).add_to(map_museum)
    total_time += e[i].X*dict_time[i][4]
    total_cost += e[i].X*dict_cost[i][4]

map_museum

In [ ]:
print("Total time: ", total_time, "/ 100")
print("Total cost: ", total_cost, "/ 400")

Total time:  58.199999999999996 / 100
Total cost:  392.5 / 400


## Half Budget:

In [ ]:
time_per_place = 5
_Time = 100
_Cost = 200


mod = gp.Model("Paranomic Tour")
mod.Params.LogToConsole = 0
location = list(df_location.index)
n = len(df_cost.keys())  #number of pairs

#add binary variables
a = mod.addVars(dict_cost.keys(), vtype="B", name = "walk")
b = mod.addVars(dict_cost.keys(), vtype="B", name = "bus")
c = mod.addVars(dict_cost.keys(), vtype="B", name = "taxi")
d = mod.addVars(dict_cost.keys(), vtype="B", name = "train")
e = mod.addVars(dict_cost.keys(), vtype="B", name = "airplane")

y = mod.addVars(dict_cost.keys(), vtype="B", name = "pathExist")
x = mod.addVars(location, vtype="B") #if visit

# Objective
mod.setObjective(gp.quicksum(df_location.loc[i]['Star Rating'] * x[i] for i in location), GRB.MAXIMIZE)

# Constraints
start = 'Hoan Kiem Lake'

# CONSTRAINTS
for i in location:
    for j in location:
        if i != j:
            mod.addConstr(y[i,j] + y[j,i] <= 1)
            # choose only 1 transportation
            mod.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] <= 1)
            # Ensure that each trip goes in one direction only
            mod.addConstr(y[i,j] + y[j,i] <= x[j])
            # y = a+b+c+d+e constraint
            mod.addConstr(y[i,j] == (a[i,j]+b[i,j]+c[i,j]+d[i,j]+e[i,j]))
            # one transportation btwn 2 locations
            mod.addConstr(y[i,j] <= 1)
            mod.addConstr(a[i,j]+b[i,j]+c[i,j]+d[i,j]+e[i,j] <= 1)

for i in location:
    if i != start:
        # x constraint
        mod.addConstr(gp.quicksum(y[i, j] for j in location if j != i) == x[i])
        mod.addConstr(gp.quicksum(y[j, i] for j in location if j != i) == x[i])
        mod.addConstr(gp.quicksum(y[i, j] for j in location if j != i) <= 1)
        mod.addConstr(gp.quicksum(y[j, i] for j in location if j != i) <= 1)

        mod.addConstr(gp.quicksum(y[i, j] for j in location if j != i) == gp.quicksum(y[j, i] for j in location if j != i))

#sum go to = leave
#mod.addConstr()

#TIME constraint
mod.addQConstr(gp.quicksum(a[i,j]*dict_time[i,j][0] + b[i,j]*dict_time[i,j][1] + c[i,j]*dict_time[i,j][2] + d[i,j]*dict_time[i,j][3] + e[i,j]*dict_time[i,j][4] for (i,j) in dict_cost.keys()) + gp.quicksum(time_per_place*x[j] for j in location) <= _Time) #24 cuz each pair counts one location twice
#COST constraint
mod.addQConstr(gp.quicksum(a[i,j]*dict_cost[i,j][0] + b[i,j]*dict_cost[i,j][1] + c[i,j]*dict_cost[i,j][2] + d[i,j]*dict_cost[i,j][3] + e[i,j]*dict_cost[i,j][4] for (i,j) in dict_cost.keys()) <= _Cost)



# Optimize
mod._vars = y
# mod._vars = a
# mod._vars = b
# mod._vars = c
# mod._vars = d
# mod._vars = e
mod.Params.lazyConstraints = 1
mod.optimize(subtourelim)


if mod.Status == gp.GRB.INFEASIBLE:
    print("Model is infeasible")
elif mod.Status == gp.GRB.UNBOUNDED:
    print("Model is unbounded")
else:
    # Print results
    print("Objective Value:", mod.ObjVal)
    #print("Time taken:", round(mod.ObjVal * 60), "hours")
    print("----------------------------------")
    print("Optimal Route:")
    tour = ['Hoan Kiem Lake']
    # a_ls = ['Hoan Kiem Lake']
    # b_ls = ['Hoan Kiem Lake']
    # c_ls = ['Hoan Kiem Lake']
    # d_ls = ['Hoan Kiem Lake']
    # e_ls = ['Hoan Kiem Lake']
    current = 'Hoan Kiem Lake'
    count = 0
    while True:
        count  += 1
        for j in location:
            if j != current and y[current, j].X > 0.5:
                tour.append(j)
                current = j
                count += 1
                break

        if current == 'Hoan Kiem Lake':
            break
        if count == 100:
          break


    print(" -> ".join(tour))
    print("----------------------------------")
    print("# of locations", count)
    #vals = mod.getAttr('x', y)
    #selected = gp.tuplelist((i, j) for i, j in dict_cost.keys() if y[i, j] > 0.5)
    # for i in range(len(tour)-1):
    #     print(tour[i], "->", tour[i+1], ":", round(dict_cost[tour[i], tour[i+1]],2))


Objective Value: 37.5
----------------------------------
Optimal Route:
Hoan Kiem Lake -> Long Bien Bridge -> Thap Tram Huong -> Notre Dame Square -> Cu Chi Tunnel -> Vung Tau Lighthouse -> Dragon Bridge -> Hoi An Ancient Town -> Hue Imperial City -> Hoan Kiem Lake
----------------------------------
# of locations 18


In [ ]:
map_museum = folium.Map(location=[16,110],
                 zoom_start= 5.5,
                 control_scale=True)

for i,row in df_location_noindex.iterrows():
    #if i != 1:
    iframe = folium.IFrame('Attraction Name:' + str(row["Tourist Attraction"]))

    #Initialise the popup using the iframe
    popup = folium.Popup(iframe, min_width=300, max_width=300)

    #Add each row to the map
    folium.Marker(location=[row['Latitude'],row['Longitude']],
                popup = popup, c=row['Tourist Attraction']).add_to(map_museum)

folium.Marker(location=[df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == 'Hoan Kiem Lake', "Latitude"].iloc[0],
                        df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == 'Hoan Kiem Lake', "Longitude"].iloc[0]],
              icon=folium.Icon(color='red', icon='pushpin')).add_to(map_museum)

points = []
for i in tour:
    points.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i, "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i, "Longitude"].iloc[0]])
#add lines
for i in tour:
    folium.PolyLine(points, color="green", weight=2.5, opacity=1).add_to(map_museum)

total_cost = 0
total_time = 0

for i in a:
  if a[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="red", weight=2.5, opacity=1).add_to(map_museum)
    total_time += a[i].X*dict_time[i][0]
    total_cost += a[i].X*dict_cost[i][0]

for i in b:
  if b[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="yellow", weight=2.5, opacity=1).add_to(map_museum)
    total_time += b[i].X*dict_time[i][1]
    total_cost += b[i].X*dict_cost[i][1]

for i in c:
  if c[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="blue", weight=2.5, opacity=1).add_to(map_museum)
    total_time += c[i].X*dict_time[i][2]
    total_cost += c[i].X*dict_cost[i][2]

for i in d:
  if d[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="pink", weight=2.5, opacity=1).add_to(map_museum)
    total_time += d[i].X*dict_time[i][3]
    total_cost += d[i].X*dict_cost[i][3]

for i in e:
  if e[i].X > 0:
    pt = []
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[0], "Longitude"].iloc[0]])
    pt.append([df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Latitude"].iloc[0],
                   df_location_noindex.loc[df_location_noindex["Tourist Attraction"] == i[1], "Longitude"].iloc[0]])
    folium.PolyLine(pt, color="cyan", weight=2.5, opacity=1).add_to(map_museum)
    total_time += e[i].X*dict_time[i][4]
    total_cost += e[i].X*dict_cost[i][4]

map_museum

In [ ]:
print("Total travel time: ", total_time, "+ 9 * 5", "/ 100")
print("Total cost: ", total_cost, "/ 200")

Total travel time:  47.0 + 9 * 5 / 100
Total cost:  186.5 / 200
